# Lab | BabyAGI with agent

**Change the planner objective below by changing the objective and the associated prompts and potential tools and agents - Wear your creativity and AI engineering hats
You can't get this wrong!**

You would need the OpenAI API KEY and the [SerpAPI KEY](https://serpapi.com/manage-api-keyhttps://serpapi.com/manage-api-key) to run this lab.


## BabyAGI with Tools

This notebook builds on top of [baby agi](baby_agi.html), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

## Install and Import Required Modules

In [1]:
!pip install langchain langchain-community langchain-experimental langchain-openai langchain-classic
!pip install -U langchain-google-genai google-generativeai

In [2]:
from typing import Optional

# Legacy chains/prompts → langchain-classic
from google import genai
from langchain_classic.chains import LLMChain
from langchain_classic.prompts import PromptTemplate

# BabyAGI still in langchain-experimental
from langchain_experimental.autonomous_agents import BabyAGI

# Gemini integrations
from langchain_google_genai import GoogleGenerativeAI

/opt/anaconda3/lib/python3.13/site-packages/pandas/core/computation/expressions.py:22: UserWarning: Pandas requires version '2.10.2' or newer of 'numexpr' (version '2.10.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [3]:
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from google import genai

In [4]:
!pip install faiss-cpu langchain-community

In [5]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore  # Updated path
from langchain_community.vectorstores import FAISS

In [6]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

GEMINI_API_KEY  = os.getenv('GEMINI_API_KEY')
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')

In [7]:
# Define your embedding model
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings_model = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001", sequence_length=3072, google_api_key=GEMINI_API_KEY)

# Initialize the vectorstore as empty
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embedding_size = 3072  # Must match the embedding dimension of the model
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(
    embedding_function=embeddings_model,  # Full embeddings object, not .embed_query
    index=index,
    docstore=InMemoryDocstore({}),
    index_to_docstore_id={}
)

## Define the Chains

BabyAGI relies on three LLM chains:
- Task creation chain to select new tasks to add to the list
- Task prioritization chain to re-prioritize tasks
- Execution Chain to execute the tasks


NOTE: in this notebook, the Execution chain will now be an agent.

In [8]:
!pip install google-search-results

In [9]:
from langchain_classic.agents import AgentExecutor, Tool, ZeroShotAgent  # Legacy agents
from langchain_classic.chains import LLMChain  # Legacy chains
from langchain_classic.prompts import PromptTemplate  # Legacy prompts
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI

todo_prompt = PromptTemplate.from_template(
"Act as a task-list generator for a database. "
    "Objective: {objective} "
    "Rules: "
    "- Output ONLY a numbered list. "
    "- Do NOT provide any conversational text, introductions, or 'Okay' messages. "
    "- The response MUST start immediately with the number '1'. "
    "Format: "
    "1. Task description "
    "2. Task description")
todo_chain = LLMChain(llm=GoogleGenerativeAI(model="gemini-3.1-pro-preview", google_api_key=GEMINI_API_KEY, temperature=0), prompt=todo_prompt)
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    ),
]


prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

/var/folders/1k/zl09cwlx1dn_f7gpczg8q5f00000gp/T/ipykernel_1700/3937933926.py:17: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use `RunnableSequence, e.g., `prompt | llm`` instead.
  todo_chain = LLMChain(llm=GoogleGenerativeAI(model="gemini-3.1-pro-preview", google_api_key=GEMINI_API_KEY, temperature=0), prompt=todo_prompt)


In [10]:
llm = GoogleGenerativeAI(model="gemini-3.1-pro-preview", google_api_key=GEMINI_API_KEY, temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

/var/folders/1k/zl09cwlx1dn_f7gpczg8q5f00000gp/T/ipykernel_1700/958427178.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the [LangGraph documentation](https://langchain-ai.github.io/langgraph/) as well as guides for [Migrating from AgentExecutor](https://python.langchain.com/docs/how_to/migrate_agent/) and LangGraph's [Pre-built ReAct agent](https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/).
  agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)


### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [11]:
OBJECTIVE = "Write a weather report for SF today"

In [12]:
# Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

In [ ]:
baby_agi({"objective": OBJECTIVE})

/var/folders/1k/zl09cwlx1dn_f7gpczg8q5f00000gp/T/ipykernel_1700/3867971396.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  baby_agi({"objective": OBJECTIVE})



*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Question: Make a todo list
Thought: I need to generate a todo list for the SF weather report.
Action: TODO
Action Input: Write a weather report for SF today

3072
